In [8]:
import pandas as pd
import json
from collections import defaultdict
import re

In [9]:
df = pd.read_csv("/Users/kkim14/tcr_structure_features/data/maura_hnncc/meta_label_sep4.csv")

/var/folders/33/b5h0gcjj46j_mgfzl_pqly3c0000gq/T/ipykernel_82277/564768651.py:1: DtypeWarning: Columns (31,35,36,112,114,117,168,170,173,174,175,176,177,178,179,180,181,182,183,184,185,202,203,204,205,206,207,208,209,210,211,212,213,214,215,216,217,218,219,220,221,222,223,224,225,226,231,258,259,260,261,262,263,264,265,266,267,268,269,273,277,278,279,289,290,292,294,297,298,299,300,301,302,303,304,305,306,307,314,315,316,317,318) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("/Users/kkim14/tcr_structure_features/data/maura_hnncc/meta_label_sep4.csv")


In [14]:
# Check columns that match the pattern
pattern = r'WholeVdj_[AB]-(fwr|cdr)[1234]'
matching_cols = [col for col in df.columns if re.match(pattern, col)]
print("Sample matching columns:", matching_cols)
print("\nSample data from first matching column:")
print(df[matching_cols[0]].head())

Sample matching columns: ['WholeVdj_A-fwr1', 'WholeVdj_B-fwr1', 'WholeVdj_A-cdr1', 'WholeVdj_B-cdr1', 'WholeVdj_A-fwr2', 'WholeVdj_B-fwr2', 'WholeVdj_A-cdr2', 'WholeVdj_B-cdr2', 'WholeVdj_A-fwr3', 'WholeVdj_B-fwr3', 'WholeVdj_A-cdr3', 'WholeVdj_B-cdr3', 'WholeVdj_A-fwr4', 'WholeVdj_B-fwr4']

Sample data from first matching column:
0     LAKTTQPISMDSYEGQEVNITCSHN
1     QQPVQSPQAVILREGEDAVINCSSS
2     GQNIDQPTEMTATEGAIVQINCTYQ
3    AQSVTQPDIHITVSEGASLELRCNYS
4     LAKTTQPISMDSYEGQEVNITCSHN
Name: WholeVdj_A-fwr1, dtype: object


In [16]:
# Check if any values contain semicolons
sample_col = df[matching_cols[0]]
has_semicolon = sample_col.astype(str).str.contains(';').any()
print(f"Contains semicolons: {has_semicolon}")
print("\nSample values with semicolons (if any):")
print(sample_col[sample_col.astype(str).str.contains(';', na=False)].head(10))

Contains semicolons: True

Sample values with semicolons (if any):
6     RKEVEQDPGPFNVPEGATVAFNCTYS;QKEVEQNSGPLSVPEGAIA...
8     AQSVTQPDIHITVSEGASLELRCNYS;AQKVTQAQTEISVVEKEDV...
9     GENVEQHPSTLSVQEGDSAVIKCTYS;GQNIDQPTEMTATEGAIVQ...
16    SQQGEEDPQALSIQEGENATMNCSYK;GIQVEQSPPDLILQEGANS...
20    DQQVKQNSPSLSVQEGRISILNCDYT;GENVEQHPSTLSVQEGDSA...
28    AQKVTQAQTEISVVEKEDVTLDCVYE;AQKVTQAQTEISVVEKEDV...
41    AQKVTQAQTEISVVEKEDVTLDCVYE;DQQVKQNSPSLSVQEGRIS...
51    SQQGEEDPQALSIQEGENATMNCSYK;GQNIDQPTEMTATEGAIVQ...
54    GEDVEQSLFLSVREGDSSVINCTYT;LAKTTQPISMDSYEGQEVNI...
70    GEDVEQSLFLSVREGDSSVINCTYT;AQSVTQLDSQVPVFEEAPVE...
Name: WholeVdj_A-fwr1, dtype: object


In [ ]:
# For each matching column, split values on ';' and keep the first element
for col in matching_cols:
    s = df[col]
    df[col] = s.apply(lambda x: x.split(';')[0] if isinstance(x, str) else x)
print("Updated matching columns to first element after ';' (if present).")

Updated matching columns to first element after ';' (if present).


In [27]:
for col in matching_cols:
    s = df[col] 
    print(f"\nNumber of NaN values in '{col}': {s.isna().sum()}")


Number of NaN values in 'WholeVdj_A-fwr1': 6158

Number of NaN values in 'WholeVdj_B-fwr1': 645

Number of NaN values in 'WholeVdj_A-cdr1': 6158

Number of NaN values in 'WholeVdj_B-cdr1': 645

Number of NaN values in 'WholeVdj_A-fwr2': 6158

Number of NaN values in 'WholeVdj_B-fwr2': 645

Number of NaN values in 'WholeVdj_A-cdr2': 6158

Number of NaN values in 'WholeVdj_B-cdr2': 645

Number of NaN values in 'WholeVdj_A-fwr3': 6158

Number of NaN values in 'WholeVdj_B-fwr3': 645

Number of NaN values in 'WholeVdj_A-cdr3': 6158

Number of NaN values in 'WholeVdj_B-cdr3': 645

Number of NaN values in 'WholeVdj_A-fwr4': 6158

Number of NaN values in 'WholeVdj_B-fwr4': 645


In [ ]:
# Remove rows with na
pre = df.shape[0]
df = df.dropna(subset=matching_cols)
print(f'filtered {pre} -> {df.shape[0]}')

filtered 53733 -> 46930


In [ ]:
AA_set = set("ARNDCEQGHILKMFPSTWYV")
print(len(AA_set))
# For each matching column, split values on ';' and keep the first element
for col in matching_cols:
    s = df[col]
    invalid_char_counts = 0
    for value in df[col].astype(str): # Ensure string type
        for char in value:
            if char not in AA_set:
                invalid_char_counts += 1
                
    print(f"\nNumber of non-appropriate AA in '{col}': {invalid_char_counts}")

20

Number of non-appropriate AA in 'WholeVdj_A-fwr1': 0

Number of non-appropriate AA in 'WholeVdj_B-fwr1': 0

Number of non-appropriate AA in 'WholeVdj_A-cdr1': 0

Number of non-appropriate AA in 'WholeVdj_B-cdr1': 0

Number of non-appropriate AA in 'WholeVdj_A-fwr2': 0

Number of non-appropriate AA in 'WholeVdj_B-fwr2': 0

Number of non-appropriate AA in 'WholeVdj_A-cdr2': 0

Number of non-appropriate AA in 'WholeVdj_B-cdr2': 0

Number of non-appropriate AA in 'WholeVdj_A-fwr3': 0

Number of non-appropriate AA in 'WholeVdj_B-fwr3': 0

Number of non-appropriate AA in 'WholeVdj_A-cdr3': 0

Number of non-appropriate AA in 'WholeVdj_B-cdr3': 0

Number of non-appropriate AA in 'WholeVdj_A-fwr4': 0

Number of non-appropriate AA in 'WholeVdj_B-fwr4': 0

Number of non-appropriate AA in 'WholeVdj_B-fwr3': 0

Number of non-appropriate AA in 'WholeVdj_A-cdr3': 0

Number of non-appropriate AA in 'WholeVdj_B-cdr3': 0

Number of non-appropriate AA in 'WholeVdj_A-fwr4': 0

Number of non-appropriat

In [32]:
a_cols = [col for col in df.columns if re.match(r'WholeVdj_A-(fwr|cdr)[1234]', col)]
b_cols = [col for col in df.columns if re.match(r'WholeVdj_B-(fwr|cdr)[1234]', col)]
print(a_cols)
print(b_cols)

['WholeVdj_A-fwr1', 'WholeVdj_A-cdr1', 'WholeVdj_A-fwr2', 'WholeVdj_A-cdr2', 'WholeVdj_A-fwr3', 'WholeVdj_A-cdr3', 'WholeVdj_A-fwr4']
['WholeVdj_B-fwr1', 'WholeVdj_B-cdr1', 'WholeVdj_B-fwr2', 'WholeVdj_B-cdr2', 'WholeVdj_B-fwr3', 'WholeVdj_B-cdr3', 'WholeVdj_B-fwr4']


In [34]:
df['tra_aa'] = df[a_cols].apply(lambda x: ''.join(x), axis=1)
df['trb_aa'] = df[b_cols].apply(lambda x: ''.join(x), axis=1)

In [49]:
df_unique = df.groupby(['tra_aa','trb_aa'])['rownames'].apply(list).reset_index()

In [50]:
df_unique = df_unique.reset_index(names=["tcr_id"])
df_unique["tcr_id"] = "hnncc_" + df_unique["tcr_id"].astype(str)
df_unique.head()

,tcr_id,tra_aa,trb_aa,rownames
0,hnncc_0,AQKITQTQPGMFVQEKEAVTLDCTYDTSDPSYGLFWYKQPSSGEMI...,DTGVSQDPRHKITKRGQNVTFRCDPISEHNRLYWYRQTLGQGPEFL...,[T12_ATAAGAGCAAGTTCTG-1]
1,hnncc_1,AQKITQTQPGMFVQEKEAVTLDCTYDTSDPSYGLFWYKQPSSGEMI...,NAGVMQNPRHLVRRRGQEARLRCSPMKGHSHVYWYRQLPEEGLKFM...,"[T68_TAGTTGGGTTAGTGGG-1, T74_TACTCGCGTTGAACTC-1]"
2,hnncc_2,AQKITQTQPGMFVQEKEAVTLDCTYDTSDPSYGLFWYKQPSSGEMI...,GAGVSQSPRYKVAKRGQDVALRCDPISGHVSLFWYQQALGQGPEFL...,"[T93_CAAGATCTCTGGCGAC-1, T93_GCCAAATGTCTGCAAT-1]"
3,hnncc_3,AQKITQTQPGMFVQEKEAVTLDCTYDTSDPSYGLFWYKQPSSGEMI...,DSGVTQTPKHLITATGQRVTLRCSPRSGDLSVYWYQQSLDQGLQFL...,[T12_ATGAGGGAGATCACGG-1]
4,hnncc_4,AQKITQTQPGMFVQEKEAVTLDCTYDTSDPSYGLFWYKQPSSGEMI...,DTGVSQNPRHKITKRGQNVTFRCDPISEHNRLYWYRQTLGQGPEFL...,"[T55_AGAATAGTCCAAAGTC-1, T55_GTGTGCGTCACGCGGT-1]"


In [ ]:
df_unique.set_index("tcr_id", inplace=True)
df_unique.to_csv("/Users/kkim14/tcr_structure_features/data/maura_hnncc/meta_label_sep4_unique_df.csv")
df_unique.to_json("/Users/kkim14/tcr_structure_features/data/maura_hnncc/meta_label_sep4_unique_df.json", orient='index', indent=4)

In [52]:
df_unique.shape

(18145, 4)